In [ ]:
hscapiurl = "https://catalogs.mast.stsci.edu/api/v0.1/hsc"
%matplotlib inline
import astropy, time, sys, os, requests, json
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO

from astropy.table import Table, join
from astropy.io import ascii
import pandas as pd
# Set page width to fill browser for longer output lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# set width for pprint
astropy.conf.max_width = 150

def hcvcone(ra,dec,radius,table="hcvsummary",release="v3",format="csv",magtype="magaper2",
            columns=None, baseurl=hscapiurl, verbose=False,
            **kw):
    """Do a cone search of the HSC catalog (including the HCV)
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return hcvsearch(table=table,release=release,format=format,magtype=magtype,
                     columns=columns,baseurl=baseurl,verbose=verbose,**data)


def hcvsearch(table="hcvsummary",release="v3",magtype="magaper2",format="csv",
              columns=None, baseurl=hscapiurl, verbose=False,
           **kw):
    """Do a general search of the HSC catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = "{}.{}".format(cat2url(table,release,magtype,baseurl=baseurl),format)
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in hcvmetadata(table,release,magtype)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

    # either get or post works
    # r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def hcvmetadata(table="hcv",release="v3",magtype="magaper2",baseurl=hscapiurl):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    url = "{}/metadata".format(cat2url(table,release,magtype,baseurl=baseurl))
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def cat2url(table="hcvsummary",release="v3",magtype="magaper2",baseurl=hscapiurl):
    """Return URL for the specified catalog and table
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns a string with the base URL for this request
    """
    checklegal(table,release,magtype)
    if table == "summary":
        url = "{baseurl}/{release}/{table}/{magtype}".format(**locals())
    else:
        url = "{baseurl}/{release}/{table}".format(**locals())
    return url


def checklegal(table,release,magtype):
    """Checks if this combination of table, release and magtype is acceptable
    
    Raises a ValueError exception if there is problem
    """
    
    releaselist = ("v2", "v3")
    if release not in releaselist:
        raise ValueError("Bad value for release (must be one of {})".format(
            ', '.join(releaselist)))
    if release=="v2":
        tablelist = ("summary", "detailed")
    else:
        tablelist = ("summary", "detailed", "propermotions", "sourcepositions",
                    "hcvsummary", "hcv")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(
            release, ", ".join(tablelist)))
    if table == "summary":
        magtypelist = ("magaper2", "magauto")
        if magtype not in magtypelist:
            raise ValueError("Bad value for magtype (must be one of {})".format(
                ", ".join(magtypelist)))


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position
    """

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)


def mastQuery(request, url='https://mast.stsci.edu/api/v0/invoke'):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object
    url (string): The service URL

    Returns the returned data content
    """
    
    # Encoding the request as a json string
    requestString = json.dumps(request)
    r = requests.post(url, data={'request': requestString})
    r.raise_for_status()
    return r.text

In [ ]:
HSCContext= "HSCv3"

%matplotlib inline
import astropy, time, sys, os, requests, json
import numpy as np

from PIL import Image
from io import BytesIO

from astropy.table import Table, join

# check that version of mastcasjobs is new enough
# we are using some features not in version 0.0.1
from pkg_resources import get_distribution
from distutils.version import StrictVersion as V
assert V(get_distribution("mastcasjobs").version) >= V('0.0.2'), """
A newer version of mastcasjobs is required.
Update mastcasjobs to current version using this command:
pip install --upgrade git+git://github.com/rlwastro/mastcasjobs@master
"""

import mastcasjobs

# Set page width to fill browser for longer output lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# set width for pprint
astropy.conf.max_width = 150

In [ ]:
import getpass
if not os.environ.get('CASJOBS_USERID'):
    os.environ['CASJOBS_USERID'] = input('Enter Casjobs UserID:')
if not os.environ.get('CASJOBS_PW'):
    os.environ['CASJOBS_PW'] = getpass.getpass('Enter Casjobs password:')

In [ ]:
def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position
    """

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)


def mastQuery(request, url='https://mast.stsci.edu/api/v0/invoke'):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object
    url (string): The service URL

    Returns the returned data content
    """
    
    # Encoding the request as a json string
    requestString = json.dumps(request)
    r = requests.post(url, data={'request': requestString})
    r.raise_for_status()
    return r.text

target = '30 dor'
ra, dec = resolve(target)
print(target,ra,dec)

<h3>Get measurements</h3>

In [ ]:
DBtable = "HCV_demo"
jobs = mastcasjobs.MastCasJobs(context="MyDB")

# drop table if it already exists
jobs.drop_table_if_exists(DBtable)

#get main information
radius = 220.0 # arcsec
query = """
select m.MatchID, m.GroupID, m.SubGroupID, m.RA, m.Dec,
   m.AutoClass, m.ExpertClass, m.NumFilters,
   f.Filter, f.FilterDetFlag, f.VarQualFlag, f.NumLC, 
   f.MeanMag, f.MeanCorrMag, f.MAD, f.Chi2
into mydb.{DBtable}
from SearchHCVMatchID({ra},{dec},{radius}) s
join HCVmatch m on m.MatchID=s.MatchID
join HCVfilter f on f.MatchID=s.MatchID
""".format(**locals())

t0 = time.time()
results = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print("Completed in {:.1f} sec".format(time.time()-t0))
print(results)

# fast retrieval using special MAST Casjobs service
download_tab = jobs.fast_table('HCV_results', verbose=True)

In [ ]:
tab = download_tab.copy()

In [ ]:
# clean up the output format
tab['MeanMag'].format = "{:.3f}"
tab['MeanCorrMag'].format = "{:.3f}"
tab['MAD'].format = "{:.4f}"
tab['Chi2'].format = "{:.4f}"
tab['RA'].format = "{:.6f}"
tab['Dec'].format = "{:.6f}"

In [ ]:
tab = tab.to_pandas().set_index('MatchID')
tab = tab[tab['Filter'].apply(lambda x: 'WFC3' in x)]

In [ ]:
# Convert to our filter definitions
tab['Filter'] = tab.Filter.apply(lambda x: x.split('WFC3_')[-1])
filters_to_keep = ['F555W', 'F814W', 'F656N', 'F336W', 'F438W']
tab = tab[tab.Filter.apply(lambda x: x in filters_to_keep)]
tab = tab.copy().set_index('Filter', append=True)

In [ ]:
tab = tab[tab.NumFilters>=4]

In [ ]:
a = tab['MeanMag'].unstack()#[(tab.MeanMag<19)&(tab.MAD>0.5)]
index = a[(a['F555W'] <22)&(a['F656N']<19)].index
tab.loc[index][tab.loc[index].MAD>.5]

In [ ]:
MatchIDs = tab.index.get_level_values(0).unique().values
MatchIDs

In [ ]:
try: 
    print('Reading from disk')
    HCV_df = pd.read_csv('phot_df_HCV.csv')
except:
    filters = ['WFC3_F438W', 'WFC3_F814W', 'WFC3_F336W', 'WFC3_F555W', 'WFC3_F656N']
    MatchIDs_chunks = np.hsplit(MatchIDs[:23700], 79)
    MatchIDs_chunks.append(MatchIDs[23700:])
    for iter_,MatchID_chunk in enumerate(MatchIDs_chunks):
        sub_df = ascii.read(hcvsearch(table='hcv',MatchID=MatchID_chunk,Filter=filters)).to_pandas()
        if iter_==0:
            HCV_df = sub_df.copy()
        else:
            HCV_df = pd.concat((HCV_df,sub_df), axis=0)
HCV_df = HCV_df.set_index(['MatchID', 'Filter'])
HCV_df = HCV_df[(HCV_df.MJD - HCV_df.MJD.median()).abs()<1000]

In [ ]:
def plot(matchid):
    data = HCV_df.loc[matchid]
    filters=data.index.get_level_values(0).unique().values
    fig,ax=plt.subplots(figsize=(10,6))
    for i in range(len(filters)):
        filter_ = filters[i]
        ax.plot(data.loc[filter_]['MJD'], data.loc[filter_]['CorrMag'], label=filter_)
        ax.scatter(data.loc[filter_]['MJD'], data.loc[filter_]['CorrMag'], c='red', s=2, label=None)
        ax.errorbar(data.loc[filter_]['MJD'], data.loc[filter_]['CorrMag'], yerr=data.loc[filter_]['MagErr'], label=None, ms=8, ls='none')
    plt.legend(loc='best')
    plt.xlabel('MJD')
    plt.ylabel('Magnitude')
    plt.show()

In [ ]:
plot(102550247)

# ring vs annulus

In [ ]:
Ha_mags = tab[tab.index.get_level_values(1)=='F656N'][['MeanCorrMag', 'AutoClass', 'MAD']].reset_index().drop('Filter', axis=1).set_index('MatchID')
#stars = Ha_mags.index.get_level_values(0).unique()
V_mags = tab[tab.index.get_level_values(1)=='F555W'][['MeanCorrMag', 'AutoClass', 'MAD']].reset_index().drop('Filter', axis=1).set_index('MatchID')
I_mags = tab[tab.index.get_level_values(1)=='F814W'][['MeanCorrMag', 'AutoClass', 'MAD']].reset_index().drop('Filter', axis=1).set_index('MatchID')

V_I = (V_mags.MeanCorrMag - I_mags.MeanCorrMag).dropna()
V_Ha = (V_mags.MeanCorrMag - Ha_mags.MeanCorrMag).dropna()

wsfvc = Ha_mags[(Ha_mags.AutoClass==1)*(Ha_mags.MAD>0.3)].index.values
wnovar = Ha_mags[Ha_mags.AutoClass==0].index.values
wmfvc = Ha_mags[(Ha_mags.AutoClass==2)*(Ha_mags.MAD>0.3)].index.values

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.figure(1,(15,10))
plt.plot(V_I.loc[wnovar], V_Ha.loc[wnovar], 'x', markersize=4, color='silver',
          label='{:,} non-variable'.format(len(wnovar)))
plt.plot(V_I.loc[wsfvc], V_Ha.loc[wsfvc], 'o', markersize=5, color='blue',
          label='{:,} single-filter variable candidates'.format(len(wsfvc)))
plt.plot(V_I.loc[wmfvc], V_Ha.loc[wmfvc], 'o', markersize=5, color='tab:cyan',
          label='{:,} multi-filter variable candidates'.format(len(wmfvc)))
plt.xlabel('V-I [mag]')
plt.ylabel('V-Ha')
plt.legend(loc='best', title='{} HSC measurements near {}'.format(len(tab),target))


In [ ]:
subtab = tab#[tab.VarQualFlag=='AAAAA'].sort_values(by='MAD')
all_stars = Table.read('../HST_Guido/30dor_all_newerr.UBVIHa.rot', format='ascii').to_pandas()
all_stars.columns = 'ID;x;y;RA;Dec;u_1;eu_2;b_1;eb_2;v_1;ev_2;i_1;ei_2;ha_1;eha_2'.split(';')
all_stars = all_stars.set_index('ID')
a = all_stars[['RA', 'Dec']]

In [ ]:
distances = np.sqrt((subtab.groupby('MatchID')['RA'].median().values - a.iloc[:,0][:,np.newaxis])**2 + (subtab.groupby('MatchID')['Dec'].median().values - a.iloc[:,1][:,np.newaxis])**2)

In [ ]:
mindist = pd.DataFrame({'MinDist':np.min(distances, axis=0), 'NR':np.argmin(distances, axis=0)+1}, index = subtab.groupby('MatchID')['RA'].median().index.values)

In [ ]:
matched_subdf = subtab.reset_index().set_index('MatchID').join(mindist, how='left').reset_index().set_index(['index', 'Filter'])

In [ ]:
matches = matched_subdf[matched_subdf.MinDist<(1/3600)].sort_values(by=['MeanCorrMag'])
#matches = matches[matches.MAD>0.2]
#matches = matches[matches.index.get_level_values(1)=='F814W']
#matches.sort_values(by='MeanCorrMag')
matches.reset_index().Filter.value_counts()

In [ ]:
all_stars.loc[15115].RA

In [ ]:
# Match our finds to this one
from astropy.wcs import WCS
from astropy.io import fits

In [ ]:
header = fits.Header.fromfile('../HST_Guido/wcs/F814W.head', padding=False, endcard=False)
hdul = fits.HDUList(fits.PrimaryHDU(header=header))

In [ ]:
ascii.read('../HST_Guido/30dor_all_newerr.UBVIHa.rot').to_pandas()[['col2', 'col3']].to_csv('xypos.dat', index=None, sep='\t', header=False)

In [ ]:
from drizzlepac import pixtosky
pixtosky.xy2rd(hdul, coordfile='xypos.dat', output='GAIA_RaDec.dat', hms=False)

In [ ]:
a=ascii.read('GAIA_RaDec.dat').to_pandas()
plt.scatter(a.iloc[:,0], a.iloc[:,1], s=1)
plt.scatter(tab.RA, tab.Dec, s=1)

In [ ]:
def get_hla_cutout(imagename,ra,dec,size=33,autoscale=99.5,asinh=True,zoom=1):
    
    """Get JPEG cutout for an image"""
    
    url = "https://hla.stsci.edu/cgi-bin/fitscut.cgi"
    r = requests.get(url, params=dict(ra=ra, dec=dec, size=size, 
            format="jpeg", red=imagename, autoscale=autoscale, asinh=asinh, zoom=zoom))
    im = Image.open(BytesIO(r.content))
    return im

In [ ]:
def plot_images(matchid, tab):
    lc = HCV_df.loc[matchid]['ImageName']
    tab = tab.loc[matchid]
    # sort images in MJD order
    ind = np.argsort(lc['MJD'])

    # we plot zoomed-in and zoomed-out views side-by-side for each selected image
    nim = len(ind)*2
    ncols = 2 # images per row
    nrows = (nim+ncols-1)//ncols

    imsize1 = 19
    imsize2 = 101
    mra = tab['RA'][0]
    mdec = tab['Dec'][0]

    pylab.rcParams.update({"font.size":14})
    pylab.figure(1,(12, (12/ncols)*nrows))
    t0 = time.time()
    ip = 0
    for k in ind:
        im1 = get_hla_cutout(lc['ImageName'][k],mra,mdec,size=imsize1)
        ip += 1
        pylab.subplot(nrows,ncols,ip)
        pylab.imshow(im1,origin="upper",cmap="gray")
        pylab.title(lc['ImageName'][k],fontsize=14)
        im2 = get_hla_cutout(lc['ImageName'][k],mra,mdec,size=imsize2)
        ip += 1
        pylab.subplot(nrows,ncols,ip)
        pylab.imshow(im2,origin="upper",cmap="gray")
        xbox = np.array([-1,1])*imsize1/2 + (imsize2-1)//2
        pylab.plot(xbox[[0,1,1,0,0]],xbox[[0,0,1,1,0]],'r-',linewidth=1)
        pylab.title('m={:.3f} MJD={:.2f}'.format(lc['CorrMag'][k],lc['MJD'][k]),fontsize=14)
        print("{:.1f} s: finished {} of {} epochs".format(time.time()-t0,ip//2,len(ind)))
    pylab.tight_layout()
    print("{:.1f} s: got {} cutouts".format(time.time()-t0,ip))

In [ ]:
plot_images(97159085, tab)